# Anomaly Detection in Traffic Signs recognition

This project aims to enhance the effectiveness of traffic sign recognition systems by incorporating anomaly detection using YOLOv8 and autoencoders.

The primary goal is to identify and flag anomalies within traffic signs, such as damaged or incorrectly placed signs, to improve road safety and the reliability of automated driving systems.

By leveraging advanced computer vision techniques, this project seeks to contribute to more accurate and robust traffic sign recognition, ultimately enhancing the safety and efficiency of transportation systems.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Object Detection using YOLOV8


### Libraries

In [ ]:
import cv2
import glob
import random
import os
import yaml

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from collections import defaultdict
import plotly.express as px
from shutil import copyfile
from PIL import Image
import numpy as np

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.1/618.1 kB 7.8 MB/s eta 0:00:00


In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))

        cv2.rectangle(
            image,
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))

    all_images.sort()

    num_images = len(all_images)

    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

In [ ]:
%%writefile traffic_signs.yaml

path: /content/drive/MyDrive/final_dataset/
train: train/images
val: test/images

# class names
names:
  - 'no_stopping'
  - 'eighty'
  - 'fifteen'
  - 'fifty'
  - 'five'
  - 'forty'
  - 'seventy'
  - 'sixty'
  - 'thirty'
  - 'twenty'


Writing traffic_signs.yaml


In [ ]:
# Directory paths for images and labels
data_dir = '/content/drive/MyDrive/final_dataset/'
image_dir_train = os.path.join(data_dir, 'train/images')
label_dir_train = os.path.join(data_dir, 'train/labels')

# Read class names from the YAML file
class_names_file = '/content/traffic_signs.yaml'
with open(class_names_file, 'r') as yaml_file:
    class_data = yaml.safe_load(yaml_file)
    class_names = class_data['names']

# Create a dictionary to store class counts
class_samples = defaultdict(list)

# Iterate through the label files for training images to collect one sample image per class
for label_filename in os.listdir(label_dir_train):
    label_file = os.path.join(label_dir_train, label_filename)
    if os.path.exists(label_file):
        with open(label_file, 'r') as label_f:
            for line in label_f:
                label_values = line.strip().split()
                if len(label_values) == 5:  # Check if the line has bounding box information
                    class_index = int(label_values[0])
                    image_filename = os.path.splitext(label_filename)[0] + '.jpg'
                    class_samples[class_index].append(image_filename)

# Determine how many classes to display in each row
max_classes_per_row = 5
num_classes = len(class_samples)
num_samples_per_class = 1

# Calculate the number of rows needed
num_rows = (num_classes + max_classes_per_row - 1) // max_classes_per_row

# Create subplots with the determined number of rows and columns
fig, axs = plt.subplots(num_rows, max_classes_per_row, figsize=(15, 3 * num_rows))

for class_index, image_list in class_samples.items():
    row = class_index // max_classes_per_row
    col = class_index % max_classes_per_row

    for sample_index in range(num_samples_per_class):
        if sample_index < len(image_list):
            image_filename = random.choice(image_list)  # Randomly select a sample image from the class
            image_path = os.path.join(image_dir_train, image_filename)
            img = plt.imread(image_path)

            # Display the image in the corresponding subplot
            axs[row, col].imshow(img)
            axs[row, col].set_title(class_names[class_index])
            axs[row, col].axis('off')

# Remove empty subplots
for i in range(num_classes, num_rows * max_classes_per_row):
    axs.flatten()[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:

# Iterate through the label files for training images to collect one sample image per class
for label_filename in os.listdir(label_dir_train):
    label_file = os.path.join(label_dir_train, label_filename)
    if os.path.exists(label_file):
        with open(label_file, 'r') as label_f:
            for line in label_f:
                label_values = line.strip().split()
                if len(label_values) == 5:  # Check if the line has bounding box information
                    class_index = int(label_values[0])
                    image_filename = os.path.splitext(label_filename)[0] + '.jpg'
                    class_samples[class_index].append(image_filename)

# Determine how many classes to display in each row
max_classes_per_row = 5
num_classes = len(class_samples)
num_samples_per_class = 1  # You can change this to display more than one image per class

# Calculate the number of rows needed
num_rows = (num_classes + max_classes_per_row - 1) // max_classes_per_row

# Create subplots with the determined number of rows and columns
fig, axs = plt.subplots(num_rows, max_classes_per_row, figsize=(15, 3 * num_rows))

for class_index, image_list in class_samples.items():
    row = class_index // max_classes_per_row
    col = class_index % max_classes_per_row

    for sample_index in range(num_samples_per_class):
        if sample_index < len(image_list):
            image_filename = random.choice(image_list)  # Randomly select a sample image from the class
            image_path = os.path.join(image_dir_train, image_filename)
            img = plt.imread(image_path)

            # Display the image in the corresponding subplot
            axs[row, col].imshow(img)
            axs[row, col].set_title(class_names[class_index])
            axs[row, col].axis('off')

# Remove empty subplots
for i in range(num_classes, num_rows * max_classes_per_row):
    axs.flatten()[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:

# Define the path to the directory containing the images
train_dir = '/content/drive/MyDrive/final_dataset/train/images'
valid_dir = '/content/drive/MyDrive/final_dataset/valid/images'
test_dir = '/content/drive/MyDrive/final_dataset/test/images'
# Use os.listdir to get the list of files in the directory
image_files1 = os.listdir(train_dir)
image_files2 = os.listdir(valid_dir)
image_files3 = os.listdir(test_dir)
# Use len() to count the number of image files
num_images1 = len(image_files1)
num_images2 = len(image_files2)
num_images3 = len(image_files3)

# Print the count
print(f"Number of images in Training set: {num_images1}")
print(f"Number of images in Validation set: {num_images2}")
print(f"Number of images in Testing set: {num_images3}")
labels = ["Training", "Validation", "Testing"]
sizes = [num_images1, num_images2, num_images3]
colors = ['gold', 'yellowgreen', 'lightcoral']
explode = (0.1, 0, 0)  # explode the 1st slice (train_dir)

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title('Distribution of Images in Each Directory')
plt.show()

In [ ]:

# Directory paths for images and labels
data_dir = '/content/drive/MyDrive/final_dataset/'
image_dir_train = os.path.join(data_dir, 'train/images')
image_dir_val = os.path.join(data_dir, 'valid/images')
image_dir_test = os.path.join(data_dir, 'test/images')
label_dir_train = os.path.join(data_dir, 'train/labels')
label_dir_val = os.path.join(data_dir, 'valid/labels')
label_dir_test = os.path.join(data_dir, 'test/labels')

# Read class names from the YAML file
class_names_file = '/content/traffic_signs.yaml'
with open(class_names_file, 'r') as yaml_file:
    class_data = yaml.safe_load(yaml_file)
    class_names = class_data['names']

# Create dictionaries to store class counts for training, validation, and testing separately
class_counts_train = defaultdict(int)
class_counts_val = defaultdict(int)
class_counts_test = defaultdict(int)

# Iterate through the label files for training images
for label_filename in os.listdir(label_dir_train):
    label_file = os.path.join(label_dir_train, label_filename)
    if os.path.exists(label_file):
        with open(label_file, 'r') as label_f:
            for line in label_f:
                label_values = line.strip().split()
                if len(label_values) == 5:  # Check if the line has bounding box information
                    class_index = int(label_values[0])
                    class_counts_train[class_names[class_index]] += 1

# Iterate through the label files for validation images
for label_filename in os.listdir(label_dir_val):
    label_file = os.path.join(label_dir_val, label_filename)
    if os.path.exists(label_file):
        with open(label_file, 'r') as label_f:
            for line in label_f:
                label_values = line.strip().split()
                if len(label_values) == 5:  # Check if the line has bounding box information
                    class_index = int(label_values[0])
                    class_counts_val[class_names[class_index]] += 1

# Iterate through the label files for testing images
for label_filename in os.listdir(label_dir_test):
    label_file = os.path.join(label_dir_test, label_filename)
    if os.path.exists(label_file):
        with open(label_file, 'r') as label_f:
            for line in label_f:
                label_values = line.strip().split()
                if len(label_values) == 5:  # Check if the line has bounding box information
                    class_index = int(label_values[0])
                    class_counts_test[class_names[class_index]] += 1

# Extract class names and counts for training, validation, and testing
classes_train = list(class_counts_train.keys())
counts_train = list(class_counts_train.values())
classes_val = list(class_counts_val.keys())
counts_val = list(class_counts_val.values())
classes_test = list(class_counts_test.keys())
counts_test = list(class_counts_test.values())

# Create separate bar charts to visualize the class distribution for training, validation, and testing
plt.figure(figsize=(18, 6))


# Training set
plt.subplot(1, 3, 1)
plt.barh(classes_train, counts_train, color='skyblue')
plt.xlabel('Number of Images')
plt.ylabel('Class Names')
plt.title('Class Distribution in Training Set')
plt.gca().invert_yaxis()

# Add labels with counts to the bars in the training set
for i, v in enumerate(counts_train):
    plt.text(v, i, str(v), va='center', color='black', fontsize=8)

# Validation set
plt.subplot(1, 3, 2)
plt.barh(classes_val, counts_val, color='lightcoral')
plt.xlabel('Number of Images')
plt.ylabel('Class Names')
plt.title('Class Distribution in Validation Set')
plt.gca().invert_yaxis()

for i, v in enumerate(counts_val):
    plt.text(v, i, str(v), va='center', color='black', fontsize=8)

# Testing set
plt.subplot(1, 3, 3)
plt.barh(classes_test, counts_test, color='lightgreen')
plt.xlabel('Number of Images')
plt.ylabel('Class Names')
plt.title('Class Distribution in Testing Set')
plt.gca().invert_yaxis()

for i, v in enumerate(counts_test):
    plt.text(v, i, str(v), va='center', color='black', fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
import os
import random
import yaml
from collections import defaultdict
import plotly.express as px

# Directory paths for images and labels
data_dir = '/content/drive/MyDrive/final_dataset/'
image_dir_train = os.path.join(data_dir, 'train/images')
image_dir_val = os.path.join(data_dir, 'test/images')
label_dir_train = os.path.join(data_dir, 'train/labels')
label_dir_val = os.path.join(data_dir, 'test/labels')

# Read class names from the YAML file
class_names_file = '/content/traffic_signs.yaml'
with open(class_names_file, 'r') as yaml_file:
    class_data = yaml.safe_load(yaml_file)
    class_names = class_data['names']

# Create a dictionary to store class counts
class_counts = defaultdict(int)

# Iterate through the label files for validation images
for label_filename in os.listdir(label_dir_val):
    label_file = os.path.join(label_dir_val, label_filename)
    if os.path.exists(label_file):
        with open(label_file, 'r') as label_f:
            for line in label_f:
                label_values = line.strip().split()
                if len(label_values) == 5:  # Check if the line has bounding box information
                    class_index = int(label_values[0])
                    class_counts[class_names[class_index]] += 1

# Convert class_counts to a DataFrame for plotly
class_counts_df = [{'class': class_name, 'count': count} for class_name, count in class_counts.items()]

# Create an interactive treemap
fig = px.treemap(class_counts_df, path=['class'], values='count', title='Class Distribution Treemap')
fig.update_traces(textinfo='label+value')
fig.show()


### Training phase

In [ ]:
EPOCHS = 300
!yolo task=detect mode=train model=yolov8n.pt imgsz=128 data=traffic_signs.yaml epochs={EPOCHS} batch=64 name=yolov8n_v8_50e

100% 6.23M/6.23M [00:00<00:00, 129MB/s]
Ultralytics YOLOv8.0.183 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=traffic_signs.yaml, epochs=300, patience=50, batch=64, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v8_50e, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=T

In [ ]:
import matplotlib.pyplot as plt

# Number of epochs and corresponding mAP values
epochs = [100, 200, 226]
map_values = [0.868, 0.884, 0.899]

# Create a line graph to visualize mAP vs. number of epochs
plt.figure(figsize=(8, 6))
plt.plot(epochs, map_values, marker='o', linestyle='-')
plt.title('mAP vs. Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('mAP')
plt.grid(True)
plt.xticks(epochs)
plt.show()


In [ ]:
!cp runs/detect/yolov8n_v8_50e/weights/best.pt /content/drive/MyDrive/final_dataset/yolov8_model.pt

### Evaluation phase

In [ ]:
!yolo task=detect mode=val model=/content/drive/MyDrive/final_dataset/yolov8_model.pt name=yolov8n_eval data=traffic_signs.yaml


Ultralytics YOLOv8.0.183 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients
100% 755k/755k [00:00<00:00, 12.5MB/s]
val: Scanning /content/drive/MyDrive/final_dataset/test/labels.cache... 230 images, 0 backgrounds, 0 corrupt: 100% 230/230 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 15/15 [00:04<00:00,  3.68it/s]
                   all        230        231       0.85      0.875      0.899      0.861
           no_stopping        230          2          1      0.924      0.995      0.895
                eighty        230         12       0.95      0.833      0.937      0.928
               fifteen        230         10       0.87          1      0.995      0.995
                 fifty        230         22      0.798      0.955      0.952      0.938
                  five        230         30      0.963      0.867      0.942      0.835
      

### Inference phase

In [ ]:
!yolo task=detect \
mode=predict \
model=/content/drive/MyDrive/final_dataset/yolov8_model.pt\
source="/content/drive/MyDrive/final_dataset/test/images" \
imgsz=128 \
name=yolov8n_v8_50e_infer1280 \
show_labels=True \
save_txt=True \
save_conf=True


Ultralytics YOLOv8.0.183 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients

image 1/230 /content/drive/MyDrive/final_dataset/test/images/00001_00001_00012_png_jpg.rf.a828911891672abce3d764e07d75d852.jpg: 128x128 1 thirty, 8.8ms
image 2/230 /content/drive/MyDrive/final_dataset/test/images/000_0001_png.rf.8e1f9b5af2ee0dd06fa23de276aa2027.jpg: 128x128 1 five, 8.5ms
image 3/230 /content/drive/MyDrive/final_dataset/test/images/000_0001_png_jpg.rf.959b4f70b4089b81ad8d1d03c1694542.jpg: 128x128 1 five, 5.9ms
image 4/230 /content/drive/MyDrive/final_dataset/test/images/000_0029_png.rf.6bee78a5cd93e8b68801b6592eefe7ee.jpg: 128x128 1 no_stopping, 1 five, 1 thirty, 5.5ms
image 5/230 /content/drive/MyDrive/final_dataset/test/images/000_1_0005_png.rf.f45d9bfbb89f1935d08b8f2e10fd19f8.jpg: 128x128 1 five, 5.6ms
image 6/230 /content/drive/MyDrive/final_dataset/test/images/000_1_0020_png.rf.d24ff5f4ab2068f2ead6cdb8e11160c0.j

In [ ]:

# Collect confidence scores from the text files
files = glob.glob('/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/labels/*.txt')
confidences = []

for f in files:
    with open(f) as file:
        for line in file:
            conf = line.split()[-1]
            confidences.append(float(conf))

# Create a histogram to visualize the distribution of confidence scores
plt.figure(figsize=(8, 6))
plt.hist(confidences, bins=30, range=(0, 1), color='skyblue', edgecolor='black')
plt.title('Distribution of Confidence Scores')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# Count the number of confidence scores less than 0.5 and greater than or equal to 0.5
count_below_0_5 = sum(1 for conf in confidences if conf < 0.5)
print(f'Number of confidence scores less than 0.5: {count_below_0_5}')


Number of confidence scores less than 0.5: 27


In [ ]:

# Define the directory paths
label_dir = '/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/labels/'
output_dir = '/content/drive/MyDrive/final_dataset/low_confidence_images/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate through label files
for label_file in glob.glob(os.path.join(label_dir, '*.txt')):
    with open(label_file, 'r') as file:
        lines = file.readlines()

    # Extract confidence values from each line
    for line in lines:
        confidence = float(line.strip().split()[-1])

        # Check if confidence is less than 0.5
        if confidence < 0.5:
            # Extract image filename from label filename
            image_file = os.path.splitext(os.path.basename(label_file))[0] + '.jpg'

            # Copy the image to the output directory
            image_path_src = os.path.join('/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/', image_file)
            image_path_dst = os.path.join(output_dir, image_file)
            copyfile(image_path_src, image_path_dst)


In [ ]:
#Low confidence images
# Get a list of image files in the output directory
image_files = glob.glob(os.path.join(output_dir, '*.jpg'))

# Shuffle the list of image files
random.shuffle(image_files)

# Display 10 random images from the directory
num_images_to_display = 10
plt.figure(figsize=(15, 8))
for i in range(num_images_to_display):
    if i < len(image_files):
        image_path = image_files[i]
        image = Image.open(image_path)
        plt.subplot(2, 5, i + 1)
        plt.imshow(image)
        plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:

# Define the directory paths
label_dir = '/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/labels/'
output_dir = '/content/drive/MyDrive/final_dataset/low_confidence_cropped_images/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Set the confidence threshold
confidence_threshold = 0.5

# Iterate through label files
for label_file in glob.glob(os.path.join(label_dir, '*.txt')):
    with open(label_file, 'r') as file:
        lines = file.readlines()

    # Extract image filename from label filename
    image_file = os.path.splitext(os.path.basename(label_file))[0] + '.jpg'
    image_path = os.path.join('/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/', image_file)

    # Load the image
    image = Image.open(image_path)
    width, height = image.size

    # Process each line in the label file
    for line in lines:
        values = line.strip().split()
        confidence = float(values[4])

        # Check if confidence is less than the threshold
        if confidence < confidence_threshold:
            # Extract bounding box coordinates
            x_center = float(values[1]) * width
            y_center = float(values[2]) * height
            box_width = float(values[3]) * width
            box_height = float(values[4]) * height

            # Calculate bounding box coordinates
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))
            x_max = int(x_center + (box_width / 2))
            y_max = int(y_center + (box_height / 2))

            # Crop the region from the image
            cropped_region = image.crop((x_min, y_min, x_max, y_max))

            # Save the cropped region to the output directory
            output_path = os.path.join(output_dir, f'cropped_{image_file}')
            cropped_region.save(output_path)


In [ ]:

# Define the directory path
output_dir = '/content/drive/MyDrive/final_dataset/low_confidence_cropped_images/'

# Get a list of image files in the output directory
image_files = glob.glob(os.path.join(output_dir, '*.jpg'))

# Shuffle the list of image files
random.shuffle(image_files)

# Display 5 random images from the directory
num_images_to_display = 3
plt.figure(figsize=(15, 6))
for i in range(num_images_to_display):
    if i < len(image_files):
        image_path = image_files[i]
        image = Image.open(image_path)
        plt.subplot(1, 5, i + 1)
        plt.imshow(image)
        plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:


# Define the directory paths
label_dir = '/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/labels/'
output_dir = '/content/drive/MyDrive/final_dataset/all_cropped_images/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Set the confidence threshold
confidence_threshold = 0.5

# Iterate through label files
for label_file in glob.glob(os.path.join(label_dir, '*.txt')):
    with open(label_file, 'r') as file:
        lines = file.readlines()

    # Extract image filename from label filename
    image_file = os.path.splitext(os.path.basename(label_file))[0] + '.jpg'
    image_path = os.path.join('/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/', image_file)

    # Load the image
    image = Image.open(image_path)
    width, height = image.size

    # Process each line in the label file
    for line in lines:
        values = line.strip().split()
        confidence = float(values[4])

        # Extract bounding box coordinates
        x_center = float(values[1]) * width
        y_center = float(values[2]) * height
        box_width = float(values[3]) * width
        box_height = float(values[4]) * height

        # Calculate bounding box coordinates
        x_min = int(x_center - (box_width / 2))
        y_min = int(y_center - (box_height / 2))
        x_max = int(x_center + (box_width / 2))
        y_max = int(y_center + (box_height / 2))

        # Crop the region from the image
        cropped_region = image.crop((x_min, y_min, x_max, y_max))

        # Save the cropped region to the output directory
        output_path = os.path.join(output_dir, f'cropped_{image_file}')
        cropped_region.save(output_path)


In [ ]:

# Define the directory path
output_dir = '/content/drive/MyDrive/final_dataset/all_cropped_images/'

# Get a list of image files in the output directory
image_files = glob.glob(os.path.join(output_dir, '*.jpg'))

# Shuffle the list of image files
random.shuffle(image_files)

# Display 5 random images from the directory
num_images_to_display = 5
plt.figure(figsize=(15, 6))
for i in range(num_images_to_display):
    if i < len(image_files):
        image_path = image_files[i]
        image = Image.open(image_path)
        plt.subplot(1, 5, i + 1)
        plt.imshow(image)
        plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:

yolo_confidence=np.average(confidences)

##grouping

In [ ]:

# Define the directory paths
label_dir = '/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/labels/'
image_dir = '/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280/'
output_dir = '/content/drive/MyDrive/final_dataset/grouped_images/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Class names based on your YAML file
class_names = [
    'no_stopping',
    'eighty',
    'fifteen',
    'fifty',
    'five',
    'forty',
    'seventy',
    'sixty',
    'thirty',
    'twenty'
]

# Iterate through label files
for label_file in glob.glob(os.path.join(label_dir, '*.txt')):
    with open(label_file, 'r') as file:
        lines = file.readlines()

    # Extract image filename from label filename
    image_file = os.path.splitext(os.path.basename(label_file))[0] + '.jpg'
    class_label = None

    # Process each line in the label file
    for line in lines:
        values = line.strip().split()
        class_label = int(values[0])

    if class_label is not None:
        # Get the class name based on the label
        class_name = class_names[class_label]

        # Create a folder for the class if it doesn't exist
        class_folder = os.path.join(output_dir, class_name)
        os.makedirs(class_folder, exist_ok=True)

        # Copy the image to the class folder
        image_src_path = os.path.join(image_dir, image_file)
        image_dst_path = os.path.join(class_folder, image_file)
        copyfile(image_src_path, image_dst_path)


In [ ]:
yolo_confidence

0.9300451182795699

In [ ]:
# Plot and visualize images in a 2x2 grid.
def visualize(result_dir, num_samples=4):
    """
    Function accepts a list of images and plots
    them in a 2x2 grid.
    """
    plt.figure(figsize=(30, 20))
    image_names = glob.glob(os.path.join(result_dir, '*.jpg'))
    random.shuffle(image_names)
    for i, image_name in enumerate(image_names):
        image = plt.imread(image_name)
        plt.subplot(2, 2, i+1)
        plt.imshow(image)
        plt.axis('off')
        if i == num_samples-1:
            break
    plt.tight_layout()
    plt.show()

In [ ]:
visualize('/content/drive/MyDrive/final_dataset/runs/detect/yolov8n_v8_50e_infer1280')

## Autoencoder

In [ ]:
import os
from operator import add
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from   torch.optim      import Adam
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import albumentations as A
from   albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KernelDensity


from tqdm import tqdm

In [ ]:
def save_checkpoint(save_dict, filename=""):
    torch.save(save_dict, filename)

def load_checkpoint(checkpoint_address, model, optimizer,verbose=False):
    model.load_state_dict(checkpoint_address["state_dict"])
    optimizer.load_state_dict(checkpoint_address["optimizer"])
    if verbose:
        print("=> Checkpoint Loaded")

### Model Definition

In [ ]:
class Block(nn.Module):
    def __init__(self, in_features, out_features, down=True):
        super().__init__()

        self.conv = nn.Sequential(
                                    nn.Conv2d(in_features, out_features, kernel_size=4,stride=2,padding=1) if down
                                    else
                                    nn.ConvTranspose2d(in_features, out_features, kernel_size=4,stride=2,padding=1),
                                    nn.LeakyReLU()
        )

    def forward(self, x):
        return self.conv(x)

class AutoEncoder(nn.Module):
    def __init__(self, in_channels=3, features=[3,64,128,256,512,1024], return_bottleneck = False):
        super().__init__()

        self.return_bottleneck = return_bottleneck
        downlayers = []
        for feature in features[1:]:
            downlayers.append(Block(in_channels,feature))
            in_channels=feature

        uplayers = []
        for feature in features[::-1][1:]:
            uplayers.append(Block(in_channels,feature,down=False))
            in_channels=feature

        self.encoder = nn.Sequential(*downlayers)
        self.decoder = nn.Sequential(*uplayers)

    def forward(self,x):
        x = self.encoder(x)

        if self.return_bottleneck:
            bottleneck = x.clone()
            return self.decoder(x), bottleneck
        else:
            return self.decoder(x)


### Defining Image Paths

In [ ]:
main_path = "/content/drive/MyDrive/yolo_data/traffic_signs/"
print(sorted(list( map(add, [main_path]*len(os.listdir(main_path)), os.listdir(main_path) ))))

['/content/drive/MyDrive/yolo_data/traffic_signs/eighty', '/content/drive/MyDrive/yolo_data/traffic_signs/fifteen', '/content/drive/MyDrive/yolo_data/traffic_signs/fifty', '/content/drive/MyDrive/yolo_data/traffic_signs/five', '/content/drive/MyDrive/yolo_data/traffic_signs/forty', '/content/drive/MyDrive/yolo_data/traffic_signs/no_stopping', '/content/drive/MyDrive/yolo_data/traffic_signs/seventy', '/content/drive/MyDrive/yolo_data/traffic_signs/sixty', '/content/drive/MyDrive/yolo_data/traffic_signs/thirty', '/content/drive/MyDrive/yolo_data/traffic_signs/twenty']


In [ ]:
list_of_image_paths = []

train_paths = []
val_paths   = []
test_paths  = []

for path in sorted(os.listdir(main_path)):
  list_of_image_paths+=sorted(list( map(add, [main_path+path+"/"]*len(os.listdir(main_path+path)), os.listdir(main_path+path) )))

  class_paths = sorted(list( map(add, [main_path+path+"/"]*len(os.listdir(main_path+path)), os.listdir(main_path+path) )))

  class_train_paths, class_valTest_paths = train_test_split(class_paths        , test_size = 0.30, random_state = 42)
  class_val_paths  , class_test_paths    = train_test_split(class_valTest_paths, test_size = 0.33, random_state = 42)

  train_paths += class_train_paths
  val_paths   += class_val_paths
  test_paths  += class_test_paths

  print(len(class_train_paths), len(class_val_paths), len(class_test_paths),"\t:",path)

list_of_image_paths = sorted(list_of_image_paths)


175 50 26 	: eighty
42 12 6 	: fifteen
109 32 16 	: fifty
98 28 15 	: five
240 69 34 	: forty
77 22 11 	: no_stopping
320 92 46 	: seventy
53 15 8 	: sixty
105 30 15 	: thirty
147 42 21 	: twenty


In [ ]:
print(len(train_paths),"+",len(val_paths),"+",len(test_paths),"=",len(list_of_image_paths))

1366 + 392 + 198 = 1956


In [ ]:
 anomaly_paths = "/content/drive/MyDrive/final_dataset/low_confidence_images/"
 anomaly_paths = sorted(list( map(add, [anomaly_paths]*len(os.listdir(anomaly_paths)), os.listdir(anomaly_paths) )))


### Writing Dataset Class

In [ ]:
from albumentations.core.transforms_interface  import ImageOnlyTransform, DualTransform,to_tuple

def make_normal(temp):
  return (temp - temp.min())/(temp.max()-temp.min())

class myNormalize(ImageOnlyTransform):
    """Normalization is applied by the formula: `img = (img - img.min) / (img.min - img.max)`
    Args:
        no args, we use the min and max of the image
    Targets:
        image
    Image types:
        uint8, float32
    """

    def __init__( self, always_apply=False, p=1.0,):
        super(myNormalize, self).__init__(always_apply, p)

    def apply(self, image, **params):
        return make_normal(image)


In [ ]:
class TrafficDataset(Dataset):
  def __init__(self, paths, transform):
    self.paths     = paths
    self.transform = transform

  def __len__(self):
    return len(self.paths)

  def __getitem__(self, i):
    image = cv2.imread(self.paths[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = make_normal(image)

    if self.transform:
      image = self.transform(image=image)['image'].float()

    return image

In [ ]:
transform_train = A.Compose([
                                A.Resize(256,256),
                                A.VerticalFlip(p=0.3),
                                A.Rotate(10,p=0.3),
                                myNormalize(),
                                ToTensorV2()
])

transform_valTest = A.Compose([
                                A.Resize(256,256),
                                myNormalize(),
                                ToTensorV2()
])

In [ ]:
train_dataset = TrafficDataset(paths = train_paths, transform = transform_train  )
val_dataset   = TrafficDataset(paths = val_paths  , transform = transform_valTest)
test_dataset  = TrafficDataset(paths = test_paths , transform = transform_valTest)

anomaly_dataset = TrafficDataset(paths = anomaly_paths, transform = transform_valTest)
all_dataset = TrafficDataset(paths = list_of_image_paths, transform = transform_train)


In [ ]:
loader = DataLoader(dataset = anomaly_dataset, batch_size = 1, shuffle=False)
try:
  for idx, (temp_image) in enumerate(loader):
    # print(temp_image.shape)
    plt.figure(figsize=(4,4))
    plt.imshow(temp_image[0].transpose(0,2).transpose(1,0))
    plt.axis('off')
    plt.title(temp_image[0].shape)
    plt.show()
except KeyboardInterrupt:
  pass

### Defining Hyperparameters

In [ ]:
batch_size    = 16
learning_rate = 1e-3
num_epochs    = 100


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_address = "/content/drive/MyDrive/yolo_data/AutoEncoder_Reconstruction.pt"
grid_address       = "/content/drive/My Drive/yolo_data/grid/Grid_"

model = AutoEncoder()
optimizer = Adam(model.parameters(), lr= learning_rate)

loss_fn = nn.L1Loss()

load_model = True

In [ ]:
model_with_btn = AutoEncoder(return_bottleneck=True)
if load_model:
  model_with_btn = model_with_btn.to(device)
  load_checkpoint(model=model_with_btn, optimizer=optimizer, checkpoint_address = torch.load(checkpoint_address), verbose=True)

=> Checkpoint Loaded


In [ ]:
if load_model:
  model = model.to(device)
  load_checkpoint(model=model, optimizer=optimizer, checkpoint_address = torch.load(checkpoint_address), verbose=True)

=> Checkpoint Loaded


In [ ]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True )
val_loader   = DataLoader(dataset = val_dataset  , batch_size = batch_size, shuffle=False)

### Training

In [ ]:
def save_grid(loader, model, loss_fn, grid_address,device):
  model = model.to(device)
  model.eval()

  samples_to_check = next(iter(loader)).to(device)
  reconstructions  = model(samples_to_check)

  plt.figure(figsize=(16,16))
  for idx in range(reconstructions.shape[0]):
    plt.subplot(4,4,idx+1)
    plt.imshow(make_normal(reconstructions[idx].transpose(0,2).transpose(1,0).detach().cpu().numpy()))
    plt.axis('off')
    plt.title(round(loss_fn(reconstructions[idx],samples_to_check[idx]).item(),6),fontsize=25)

  plt.savefig(grid_address)
  plt.close()

  model.eval()


In [ ]:
def train_model(loader,model,optimizer,loss_fn,device):
  model = model.to(device=device)
  total_loss = 0

  loop = tqdm(loader)

  for batch_idx, (image) in enumerate(loop):
      image = image.to(device)

      reconstructed_image = model(image)

      loss = loss_fn(image, reconstructed_image)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      loop.set_postfix(loss = total_loss/(batch_idx+1))

  return total_loss/(batch_idx+1)

def validate_model(loader,model,loss_fn,device):
  model = model.to(device)
  total_loss = 0

  loop = tqdm(loader)

  model.eval()
  with torch.no_grad():
    for batch_idx, (image) in enumerate(loop):
        image = image.to(device)

        reconstructed_image = model(image)

        loss = loss_fn(image, reconstructed_image)

        total_loss += loss.item()
        loop.set_postfix(loss = total_loss/(batch_idx+1))

  model.train()
  return total_loss/(batch_idx+1)

def trainer(
    epochs,
    checkpoint_address,
    grid_address,
    best_score,
    training_metrics,
    train_loader,
    val_loader,
    model,
    optimizer,
    loss_fn,
    device,
    resume = False
):
    model = model.to(device)

    if resume:
        load_dict        = torch.load(checkpoint_address)
        best_score       = load_dict['score']
        training_metrics = load_dict['metrics']

    try:
        for epoch in range(len(training_metrics),epochs):
            t_loss = train_model(
                                    loader    = train_loader,
                                    model     = model,
                                    optimizer = optimizer,
                                    loss_fn   = loss_fn,
                                    device    = device
                                )
            v_loss = validate_model(
                                      loader    = val_loader,
                                      model     = model,
                                      loss_fn   = loss_fn,
                                      device    = device
                                   )

            print("Epoch: ",epoch+1)
            print("Training Loss:", t_loss )
            print("Validation Loss:", v_loss)

            training_metrics.append([t_loss,v_loss])

            image_to_check = next(iter(val_loader))[0].unsqueeze(0).to(device)

            model.eval()
            with torch.no_grad():
                reconstructed = model(image_to_check)
            model.train()

            image_to_check = image_to_check
            reconstructed  = reconstructed
            temp_loss      = round(loss_fn(image_to_check,reconstructed).item(),4)

            plt.figure(figsize=(20,5))
            plt.subplot(1,3,1)
            plt.plot(np.array(training_metrics)[:,0])
            plt.plot(np.array(training_metrics)[:,1])
            plt.legend(["train","val"])
            plt.title('Loss')

            plt.subplot(1,3,2)
            plt.imshow(image_to_check.squeeze().transpose(0,2).transpose(1,0).detach().cpu().numpy())
            plt.axis('off')
            plt.title("Original")

            plt.subplot(1,3,3)
            plt.imshow(make_normal(reconstructed.squeeze().transpose(0,2).transpose(1,0).detach().cpu().numpy()))
            plt.axis('off')
            plt.title(f"Reconstructed: {str(temp_loss)}")
            plt.show()

            save_grid(model=model,
                      loader=val_loader,
                      loss_fn=loss_fn,
                      grid_address=grid_address+str(epoch).zfill(4),
                      device=device)

            if v_loss < best_score:
                print("Saving Model!")
                best_score = v_loss
                save_dict  = {
                                  'state_dict' : model.state_dict(),
                                  'optimizer'  : optimizer.state_dict(),
                                  'score'      : best_score,
                                  'metrics'    : training_metrics
                            }

                save_checkpoint(save_dict,filename = checkpoint_address)

    except KeyboardInterrupt:
        print("Training Stopped Manually!")



In [ ]:
trainer(
    epochs              = num_epochs,
    checkpoint_address  = checkpoint_address,
    grid_address        = grid_address,
    best_score          = 100,
    training_metrics    = [],
    train_loader        = train_loader,
    val_loader          = val_loader,
    model               = model,
    optimizer           = optimizer,
    loss_fn             = loss_fn,
    device              = device
)

  0%|          | 0/86 [00:00<?, ?it/s]


NameError: ignored

### Test Inference

In [ ]:
train_loader_batchsize_1 = DataLoader(dataset = train_dataset  , batch_size = 1, shuffle=True )
val_loader_batchsize_1   = DataLoader(dataset = val_dataset    , batch_size = 1, shuffle=False)
test_loader              = DataLoader(dataset = test_dataset   , batch_size = 1, shuffle=False)
anomaly_loader           = DataLoader(dataset = anomaly_dataset, batch_size = 1, shuffle=False)

In [ ]:

def get_list_of_latents(loader, model, device):
    model = model.to(device)
    loop = tqdm(loader)

    list_of_latents = []
    for idx, (image) in enumerate(loop):
        image = image.to(device)

        _, latent = model(image)

        list_of_latents.append(latent.flatten().detach().cpu().numpy())

    return list_of_latents



In [ ]:
list_of_latents_train = get_list_of_latents(loader = train_loader_batchsize_1, model = model_with_btn, device = device)
kde = KernelDensity(kernel="gaussian", bandwidth=0.2).fit(list_of_latents_train)

100%|██████████| 1366/1366 [00:23<00:00, 57.40it/s]


In [ ]:
def get_recon_loss_and_density(loader, model, loss_fn, kde, device):
    model = model.to(device)
    total_loss    = []
    total_density = []
    loop = tqdm(loader)

    model.eval()
    with torch.no_grad():
      for batch_idx, (image) in enumerate(loop):
          image = image.to(device)
          reconstructed, latent = model(image)
          loss    = loss_fn(image, reconstructed)
          density = kde.score_samples(latent.flatten(start_dim=1).detach().cpu().numpy())[0]

          total_loss.append(loss.item())
          total_density.append(density)
          loop.set_postfix(loss = loss)

    model.train()
    return total_loss, total_density



In [ ]:
total_loss        , total_density         =  get_recon_loss_and_density(loader = val_loader_batchsize_1, model = model_with_btn, loss_fn = loss_fn, kde = kde, device=device)
total_loss_anomaly, total_density_anomaly =  get_recon_loss_and_density(loader = anomaly_loader        , model = model_with_btn, loss_fn = loss_fn, kde = kde, device=device)

100%|██████████| 71/71 [00:14<00:00,  4.95it/s, loss=tensor(0.0551, device='cuda:0')]


In [ ]:
print("Normal Images")
print("\tDensity Mean\t\t:"             , np.mean(total_density))
print("\tDensity SD  \t\t:"             , np.std(total_density))
print("\tReconstruction Loss Mean:"     , np.mean(total_loss))
print("\tReconstruction Loss SD  :"     , np.std(total_loss))

print("Anomaly Images")
print("\tDensity Mean\t\t:"        , np.mean(total_density_anomaly))
print("\tDensity SD  \t\t:"        , np.std(total_density_anomaly))
print("\tReconstruction Loss Mean:", np.mean(total_loss_anomaly))
print("\tReconstruction Loss SD  :", np.std(total_loss_anomaly))

Normal Images
	Density Mean		: 43000.00550364287
	Density SD  		: 3245.393880222268
	Reconstruction Loss Mean: 0.015278786530109997
	Reconstruction Loss SD  : 0.00878220650556563
Anomaly Images
	Density Mean		: 40251.45357854997
	Density SD  		: 4408.7365934820045
	Reconstruction Loss Mean: 0.03585165369153862
	Reconstruction Loss SD  : 0.016070802715882418


In [ ]:
density_threshold    = np.mean(total_density)
recon_loss_threshold = np.mean(total_loss_anomaly)


In [ ]:
def check_anomaly(loader, model, loss_fn, device, density_thres, recon_loss_thres, make_plot = True):
    model = model.to(device)
    loop = tqdm(loader)
    true_count = 0
    false_count = 0
    model.eval()
    with torch.no_grad():
      for batch_idx, (image) in enumerate(loop):
          image = image.to(device)
          reconstructed, latent = model(image)
          loss    = loss_fn(image, reconstructed)
          density = kde.score_samples(latent.flatten(start_dim=1).detach().cpu().numpy())[0]

          if  density < density_thres or loss > recon_loss_thres:
          #if  density < density_thres:
          #if  loss > recon_loss_thres:
              flag = True
              true_count +=1

          else:
              flag = False
              false_count +=1

          if make_plot:
              plt.figure()
              plt.subplot(1,2,1)
              plt.imshow(image.squeeze().transpose(0,2).transpose(0,1).detach().cpu())
              plt.axis('off')
              plt.title("Original")

              plt.subplot(1,2,2)
              plt.imshow(reconstructed.squeeze().transpose(0,2).transpose(0,1).detach().cpu())
              plt.axis('off')
              plt.title(f"l={round(loss.item(),6)} | d={density} | anomaly={flag}")
              plt.show()
    print()
    print(true_count,false_count)



### using density and loss

In [ ]:
check_anomaly(loader = train_loader_batchsize_1, model = model_with_btn, loss_fn = loss_fn, device = device, density_thres = density_threshold, recon_loss_thres = recon_loss_threshold, make_plot = False)

100%|██████████| 1366/1366 [04:09<00:00,  5.46it/s]


289 1077


In [ ]:
check_anomaly(loader = test_loader, model = model_with_btn, loss_fn = loss_fn, device = device, density_thres = density_threshold, recon_loss_thres = recon_loss_threshold, make_plot = False)

100%|██████████| 198/198 [01:24<00:00,  2.35it/s]


70 128


In [ ]:
check_anomaly(loader = anomaly_loader, model = model_with_btn, loss_fn = loss_fn, device = device, density_thres = density_threshold, recon_loss_thres = recon_loss_threshold, make_plot = False)

100%|██████████| 71/71 [00:19<00:00,  3.68it/s]


53 18


In [ ]:
#old results
#Using Density & Loss
#                       Actual Positive  Actual Negative
# Predicted Positive :      1077                    289
# Predicted Negative :      139                     59

#Using Density Only
#          TP  FP
# Anomaly: 1096 270
# Normal : 142 56

#Using Loss Only
#          TP  FP
# Anomaly: 1076 290
# Normal : 125 105

In [ ]:
#FN(ANOMALY= TRUE)
# TN (ANOMALY = FALSE)
check_anomaly(loader = anomaly_loader, model = model_with_btn, loss_fn = loss_fn, device = device, density_thres = 35123.12160878582, recon_loss_thres = 0.02750538301964601)

In [ ]:
#TP(ANOMALY= FALSE)
# FP (ANOMALY = TRUE)


check_anomaly(loader = test_loader, model = model_with_btn, loss_fn = loss_fn, device = device, density_thres = 35123.12160878582, recon_loss_thres = 0.02750538301964601)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Provided TP and FP values
anomaly_tp = [201,142, 125]
anomaly_fp = [29, 88, 105]
methods = ['Density & Loss', 'Density Only', 'Loss Only']

# Create confusion matrix data
confusion_matrix_data = np.array([anomaly_tp, anomaly_fp])

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_data, annot=True, fmt='d', cmap='Blues', xticklabels=methods, yticklabels=['Anomaly (TP)', 'Normal (FP)'])
plt.xlabel('Methods')
plt.ylabel('True/False Positive')
plt.title('Confusion Matrix Heatmap')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Confusion matrix values
confusion_matrix_values = np.array([[1076, 290],
                                    [125, 105]])
#                       Actual Positive  Actual Negative
# Predicted Positive :      1096                    270
# Predicted Negative :      125                     105
# Normal : 142 56
# Labels for rows and columns
row_labels = ['Predicted Positive', 'Predicted Negative']
col_labels = ['Actual Positive', 'Actual Negative']

# Create the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_values, annot=True, fmt='d', cmap='Blues', xticklabels=col_labels, yticklabels=row_labels)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Confusion Matrix Heatmap using Loss')
plt.show()
